In [1]:
import boto3
from IPython.display import clear_output, Image, display, HTML
import numpy as np
import cv2
import base64
from bokeh.plotting import figure
from bokeh.io import output_notebook, show, push_notebook
import time
import json
import os
output_notebook()

Loading BokehJS ...

In [2]:
STREAM_NAME = "pi4-001"
kvs = boto3.client("kinesisvideo")
# Grab the endpoint from GetDataEndpoint
endpoint = kvs.get_data_endpoint(
    APIName="GET_HLS_STREAMING_SESSION_URL",
    StreamName=STREAM_NAME
)['DataEndpoint']
print(endpoint)

https://b-647daf39.kinesisvideo.us-west-2.amazonaws.com


In [3]:
class VideoPlayer(object):
    def __init__(self):
        self._init = False
        self._myImage = None
        
    def __call__(self, frame):
        if frame is None:
            return
        if self._init is False:
            self.init_display(frame)
            self._init = True
        else:
            self.update_display(frame)

    def init_display(self, frame):
        assert frame is not None
        frame=cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA) # because Bokeh expects a RGBA image
        frame=cv2.flip(frame, 0) # because Bokeh flips vertically
        width=frame.shape[1]
        height=frame.shape[0]
        p = figure(x_range=(0,width), y_range=(0,height), output_backend="webgl", width=width, height=height)
        self._myImage = p.image_rgba(image=[frame], x=0, y=0, dw=width, dh=height)
        show(p, notebook_handle=True)
    
    def update_display(self, frame):
        assert frame is not None
        frame=cv2.cvtColor(frame, cv2.COLOR_BGR2RGBA)
        frame=cv2.flip(frame, 0) 
        self._myImage.data_source.data['image']=[frame]
        push_notebook()

In [4]:
# Grab the HLS Stream URL from the endpoint
kvam = boto3.client("kinesis-video-archived-media", endpoint_url=endpoint)
url = kvam.get_hls_streaming_session_url(
    StreamName=STREAM_NAME,
    PlaybackMode="LIVE"
)['HLSStreamingSessionURL']
vcap = cv2.VideoCapture(url)
(major_ver, minor_ver, subminor_ver) = (cv2.__version__).split('.')
if int(major_ver)  < 3 :
    fps = vcap.get(cv2.cv.CV_CAP_PROP_FPS)
    print("Frames per second using video.get(cv2.cv.CV_CAP_PROP_FPS): {}".format(fps))
else :
    fps = vcap.get(cv2.CAP_PROP_FPS)
    print("Frames per second using video.get(cv2.CAP_PROP_FPS) : {}".format(fps))

save_fps = 1 # save 1 frame per second
skip = int(np.ceil(fps / save_fps))
player = VideoPlayer()
cnt = 0
save_cnt = 0
max_save_cnt = 10
save_dir = os.path.expanduser('~/SageMaker/mini-amazon-go/images/{}'.format(time.time()))
print(save_dir)
os.makedirs(save_dir)

while(True):
    # Capture frame-by-frame
    read_start = time.time()
    ret, frame = vcap.read()
    read_end = time.time()
    # print('read time:', read_end-read_start)

    if frame is not None:
        start = time.time()
        frame = cv2.flip(frame, -1)
        
        # Display the resulting frame
        if cnt % skip == 0:
            cv2.imwrite(os.path.join(save_dir, str(cnt) + '.jpg'), frame)
            save_cnt += 1
            if save_cnt > max_save_cnt:
                break
            cv2.putText(frame, 'saved', (40, 40), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.8, thickness=2, color=(255, 255, 255))
        player(frame)
        cnt += 1
    else:
        print("Frame is None")
        break

# When everything done, release the capture
vcap.release()

zip_file = save_dir + '.zip'

!zip -r -j $zip_file $save_dir
print('saved zip file as:', zip_file)

Frames per second using video.get(cv2.CAP_PROP_FPS) : 15.0
/home/ec2-user/SageMaker/mini-amazon-go/images/1573605560.295764


  adding: 15.jpg (deflated 0%)
  adding: 45.jpg (deflated 0%)
  adding: 135.jpg (deflated 0%)
  adding: 60.jpg (deflated 1%)
  adding: 105.jpg (deflated 0%)
  adding: 150.jpg (deflated 0%)
  adding: 30.jpg (deflated 0%)
  adding: 120.jpg (deflated 1%)
  adding: 0.jpg (deflated 1%)
  adding: 90.jpg (deflated 0%)
  adding: 75.jpg (deflated 0%)
saved zip file as: /home/ec2-user/SageMaker/mini-amazon-go/images/1573605560.295764.zip
